# **MultilatSensorNet Performance Analysis**

# **Setup**

In [1]:
!pip install plotly

In [28]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# **Trajectory Analysis**

---
**DataFrames Creation**

In [42]:
tracked_lin_df = pd.read_csv("../data/run_linear.csv", sep=';')
true_lin_df = pd.read_csv("../data/target_linear_trajectory.csv", sep=';')
tracked_circ_df = pd.read_csv("../data/run_circular.csv", sep=';')
true_circ_df = pd.read_csv("../data/target_circular_trajectory.csv", sep=';')
sensors_pos_df = pd.read_csv("../data/sensors_pos.csv", sep=';')

---
**Target and Tracked Trajectories 3D Plot**

In [44]:
def plot_trajectories(true_df: pd.DataFrame, tracked_df: pd.DataFrame, sensors_pos_df: pd.DataFrame) -> None:
    """Plots 3D trajectories for true and tracked positions along with sensor positions.

    This function visualizes the true trajectory, tracked trajectory, and sensor positions in a 3D space
    using Plotly. It is useful for comparing the actual and estimated paths and observing the distribution of sensors.

    Args:
        true_df: A pandas DataFrame containing true trajectory positions with columns 'X', 'Y', and 'Z'.
        tracked_df: A pandas DataFrame containing tracked trajectory positions with columns 'X', 'Y', and 'Z'.
        sensors_pos_df: A pandas DataFrame containing sensor positions with columns 'X', 'Y', and 'Z'.
    """
    # Creates the figure object
    fig = go.Figure()

    # Adds tracker trajectory in red
    fig.add_trace(go.Scatter3d(
        x=tracked_df['X'], y=tracked_df['Y'], z=tracked_df['Z'],
        mode='lines',
        name='Tracker Verbose off no var Trajectory',
        line=dict(color='red')
    ))

    # Plots sensors in the 3D space with green markers
    fig.add_trace(go.Scatter3d(
        x=sensors_pos_df['X'], y=sensors_pos_df['Y'], z=sensors_pos_df['Z'],
        mode='markers',
        name='Sensors',
        marker=dict(color='green', size=5, symbol='circle')
    ))

    # Adds true trajectory in blue
    fig.add_trace(go.Scatter3d(
        x=true_df['X'], y=true_df['Y'], z=true_df['Z'],
        mode='lines',
        name='True Trajectory',
        line=dict(color='blue')
    ))

    # Updates the layout with labels and title
    fig.update_layout(
        title='Target and Tracked 3D Trajectories',
        scene=dict(
            xaxis_title='X axis',
            yaxis_title='Y axis',
            zaxis_title='Z axis',
            xaxis=dict(range=[0, 4.5]),
            yaxis=dict(range=[0, 4.5]),
            zaxis=dict(range=[0, 2.75])
        )
    )

    # Shows the plot
    fig.show()

In [45]:
plot_trajectories(true_lin_df, tracked_lin_df, sensors_pos_df)
plot_trajectories(true_circ_df, tracked_circ_df, sensors_pos_df)

---
**Residual Analysis**

In [52]:
def compute_residuals(true_df: pd.DataFrame, tracked_df: pd.DataFrame) -> pd.DataFrame:
    """Computes residuals between true and tracked positions.

    This function calculates the difference between true and tracked positions for each coordinate
    ('X', 'Y', 'Z') and returns the residuals as a new DataFrame.

    Args:
        true_df: A pandas DataFrame containing true trajectory positions with columns 'X', 'Y', and 'Z'.
        tracked_df: A pandas DataFrame containing tracked trajectory positions with columns 'X', 'Y', and 'Z'.

    Returns:
        A pandas DataFrame containing residuals for each coordinate ('X', 'Y', 'Z').
    """
    residuals = true_df.values - tracked_df.values
    residual_df = pd.DataFrame(residuals, columns=['X', 'Y', 'Z'])

    return residual_df

residual_df = compute_residuals(true_lin_df, tracked_lin_df)
residual_df.describe()

,X,Y,Z
count,100.000000,100.000000,100.000000
mean,0.000040,0.000030,0.000170
std,0.001483,0.001374,0.002198
min,-0.004000,-0.003000,-0.005000
25%,-0.001000,-0.001000,-0.001250
50%,0.000000,0.000000,0.000000
75%,0.001000,0.001000,0.002000
max,0.005000,0.003000,0.005000


In [53]:
residual_df = compute_residuals(true_circ_df, tracked_circ_df)
residual_df.describe()

,X,Y,Z
count,100.000000,100.000000,100.000000
mean,-0.038510,-0.004390,-0.000490
std,0.051141,0.066435,0.009489
min,-0.128000,-0.124000,-0.063000
25%,-0.088500,-0.041000,-0.002000
50%,-0.005000,0.001000,0.000000
75%,0.000000,0.004250,0.002000
max,0.057000,0.133000,0.039000


In [54]:
def compute_euclidean_distances(true_df: pd.DataFrame, tracked_df: pd.DataFrame) -> pd.DataFrame:
    """Computes Euclidean distances between true and tracked positions.

    This function calculates the Euclidean distance between corresponding points in the true and tracked
    data frames for each observation and returns the distances as a new DataFrame.

    Args:
        true_df: A pandas DataFrame containing true trajectory positions with columns 'X', 'Y', and 'Z'.
        tracked_df: A pandas DataFrame containing tracked trajectory positions with columns 'X', 'Y', and 'Z'.

    Returns:
        A pandas DataFrame containing distances for each point.
    """
    distances = np.sqrt(
        (true_df['X'] - tracked_df['X'])**2 +
        (true_df['Y'] - tracked_df['Y'])**2 +
        (true_df['Z'] - tracked_df['Z'])**2
    )
    distances_df = pd.DataFrame(distances, columns=['Distance'])

    return distances_df

distances_df = compute_euclidean_distances(true_lin_df, tracked_lin_df)
distances_df.describe()

,Distance
count,100.000000
mean,0.002715
std,0.001225
min,0.000000
25%,0.001732
50%,0.002449
75%,0.003499
max,0.005745


In [55]:
distances_df = compute_euclidean_distances(true_circ_df, tracked_circ_df)
distances_df.describe()

,Distance
count,100.000000
mean,0.070521
std,0.060111
min,0.001000
25%,0.003533
50%,0.121484
75%,0.127031
max,0.133555
